In [1]:
# look at tools/set_up_magics.ipynb
yandex_metrica_allowed = True ; get_ipython().run_cell('# one_liner_str\n\nget_ipython().run_cell_magic(\'javascript\', \'\', \'// setup cpp code highlighting\\nIPython.CodeCell.options_default.highlight_modes["text/x-c++src"] = {\\\'reg\\\':[/^%%cpp/]} ;\')\n\n# creating magics\nfrom IPython.core.magic import register_cell_magic, register_line_magic\nfrom IPython.display import display, Markdown, HTML\nimport argparse\nfrom subprocess import Popen, PIPE\nimport random\nimport sys\nimport os\nimport re\nimport signal\nimport shutil\nimport shlex\nimport glob\n\n@register_cell_magic\ndef save_file(args_str, cell, line_comment_start="#"):\n    parser = argparse.ArgumentParser()\n    parser.add_argument("fname")\n    parser.add_argument("--ejudge-style", action="store_true")\n    args = parser.parse_args(args_str.split())\n    \n    cell = cell if cell[-1] == \'\\n\' or args.no_eof_newline else cell + "\\n"\n    cmds = []\n    with open(args.fname, "w") as f:\n        f.write(line_comment_start + " %%cpp " + args_str + "\\n")\n        for line in cell.split("\\n"):\n            line_to_write = (line if not args.ejudge_style else line.rstrip()) + "\\n"\n            if line.startswith("%"):\n                run_prefix = "%run "\n                if line.startswith(run_prefix):\n                    cmds.append(line[len(run_prefix):].strip())\n                    f.write(line_comment_start + " " + line_to_write)\n                    continue\n                run_prefix = "%# "\n                if line.startswith(run_prefix):\n                    f.write(line_comment_start + " " + line_to_write)\n                    continue\n                raise Exception("Unknown %%save_file subcommand: \'%s\'" % line)\n            else:\n                f.write(line_to_write)\n        f.write("" if not args.ejudge_style else line_comment_start + r" line without \\n")\n    for cmd in cmds:\n        display(Markdown("Run: `%s`" % cmd))\n        get_ipython().system(cmd)\n\n@register_cell_magic\ndef cpp(fname, cell):\n    save_file(fname, cell, "//")\n\n@register_cell_magic\ndef asm(fname, cell):\n    save_file(fname, cell, "//")\n    \n@register_cell_magic\ndef makefile(fname, cell):\n    assert not fname\n    save_file("makefile", cell.replace(" " * 4, "\\t"))\n        \n@register_line_magic\ndef p(line):\n    try:\n        expr, comment = line.split(" #")\n        display(Markdown("`{} = {}`  # {}".format(expr.strip(), eval(expr), comment.strip())))\n    except:\n        display(Markdown("{} = {}".format(line, eval(line))))\n        \ndef show_file(file, clear_at_begin=True, return_html_string=False):\n    if clear_at_begin:\n        get_ipython().system("truncate --size 0 " + file)\n    obj = file.replace(\'.\', \'_\').replace(\'/\', \'_\') + "_obj"\n    html_string = \'\'\'\n        <!--MD_BEGIN_FILTER-->\n        <script type=text/javascript>\n        var entrance___OBJ__ = 0;\n        var errors___OBJ__ = 0;\n        function refresh__OBJ__()\n        {\n            entrance___OBJ__ -= 1;\n            var elem = document.getElementById("__OBJ__");\n            if (elem) {\n                var xmlhttp=new XMLHttpRequest();\n                xmlhttp.onreadystatechange=function()\n                {\n                    var elem = document.getElementById("__OBJ__");\n                    console.log(!!elem, xmlhttp.readyState, xmlhttp.status, entrance___OBJ__);\n                    if (elem && xmlhttp.readyState==4) {\n                        if (xmlhttp.status==200)\n                        {\n                            errors___OBJ__ = 0;\n                            if (!entrance___OBJ__) {\n                                elem.innerText = xmlhttp.responseText;\n                                entrance___OBJ__ += 1;\n                                console.log("req");\n                                window.setTimeout("refresh__OBJ__()", 300); \n                            }\n                            return xmlhttp.responseText;\n                        } else {\n                            errors___OBJ__ += 1;\n                            if (errors___OBJ__ < 10 && !entrance___OBJ__) {\n                                entrance___OBJ__ += 1;\n                                console.log("req");\n                                window.setTimeout("refresh__OBJ__()", 300); \n                            }\n                        }\n                    }\n                }\n                xmlhttp.open("GET", "__FILE__", true);\n                xmlhttp.setRequestHeader("Cache-Control", "no-cache");\n                xmlhttp.send();     \n            }\n        }\n        \n        if (!entrance___OBJ__) {\n            entrance___OBJ__ += 1;\n            refresh__OBJ__(); \n        }\n        </script>\n        \n        <font color="white"> <tt>\n        <p id="__OBJ__" style="font-size: 16px; border:3px #333333 solid; background: #333333; border-radius: 10px; padding: 10px;  "></p>\n        </tt> </font>\n        <!--MD_END_FILTER-->\n        <!--MD_FROM_FILE __FILE__ -->\n        \'\'\'.replace("__OBJ__", obj).replace("__FILE__", file)\n    if return_html_string:\n        return html_string\n    display(HTML(html_string))\n    \nBASH_POPEN_TMP_DIR = "./bash_popen_tmp"\n    \ndef bash_popen_terminate_all():\n    for p in globals().get("bash_popen_list", []):\n        print("Terminate pid=" + str(p.pid), file=sys.stderr)\n        p.terminate()\n    globals()["bash_popen_list"] = []\n    if os.path.exists(BASH_POPEN_TMP_DIR):\n        shutil.rmtree(BASH_POPEN_TMP_DIR)\n\nbash_popen_terminate_all()  \n\ndef bash_popen(cmd):\n    if not os.path.exists(BASH_POPEN_TMP_DIR):\n        os.mkdir(BASH_POPEN_TMP_DIR)\n    h = os.path.join(BASH_POPEN_TMP_DIR, str(random.randint(0, 1e18)))\n    stdout_file = h + ".out.html"\n    stderr_file = h + ".err.html"\n    run_log_file = h + ".fin.html"\n    \n    stdout = open(stdout_file, "wb")\n    stdout = open(stderr_file, "wb")\n    \n    html = """\n    <table width="100%">\n    <colgroup>\n       <col span="1" style="width: 70px;">\n       <col span="1">\n    </colgroup>    \n    <tbody>\n      <tr> <td><b>STDOUT</b></td> <td> {stdout} </td> </tr>\n      <tr> <td><b>STDERR</b></td> <td> {stderr} </td> </tr>\n      <tr> <td><b>RUN LOG</b></td> <td> {run_log} </td> </tr>\n    </tbody>\n    </table>\n    """.format(\n        stdout=show_file(stdout_file, return_html_string=True),\n        stderr=show_file(stderr_file, return_html_string=True),\n        run_log=show_file(run_log_file, return_html_string=True),\n    )\n    \n    cmd = """\n        bash -c {cmd} &\n        pid=$!\n        echo "Process started! pid=${{pid}}" > {run_log_file}\n        wait ${{pid}}\n        echo "Process finished! exit_code=$?" >> {run_log_file}\n    """.format(cmd=shlex.quote(cmd), run_log_file=run_log_file)\n    # print(cmd)\n    display(HTML(html))\n    \n    p = Popen(["bash", "-c", cmd], stdin=PIPE, stdout=stdout, stderr=stdout)\n    \n    bash_popen_list.append(p)\n    return p\n\n\n@register_line_magic\ndef bash_async(line):\n    bash_popen(line)\n    \n    \ndef show_log_file(file, return_html_string=False):\n    obj = file.replace(\'.\', \'_\').replace(\'/\', \'_\') + "_obj"\n    html_string = \'\'\'\n        <!--MD_BEGIN_FILTER-->\n        <script type=text/javascript>\n        var entrance___OBJ__ = 0;\n        var errors___OBJ__ = 0;\n        function halt__OBJ__(elem, color)\n        {\n            elem.setAttribute("style", "font-size: 14px; background: " + color + "; padding: 10px; border: 3px; border-radius: 5px; color: white; ");                    \n        }\n        function refresh__OBJ__()\n        {\n            entrance___OBJ__ -= 1;\n            if (entrance___OBJ__ < 0) {\n                entrance___OBJ__ = 0;\n            }\n            var elem = document.getElementById("__OBJ__");\n            if (elem) {\n                var xmlhttp=new XMLHttpRequest();\n                xmlhttp.onreadystatechange=function()\n                {\n                    var elem = document.getElementById("__OBJ__");\n                    console.log(!!elem, xmlhttp.readyState, xmlhttp.status, entrance___OBJ__);\n                    if (elem && xmlhttp.readyState==4) {\n                        if (xmlhttp.status==200)\n                        {\n                            errors___OBJ__ = 0;\n                            if (!entrance___OBJ__) {\n                                if (elem.innerHTML != xmlhttp.responseText) {\n                                    elem.innerHTML = xmlhttp.responseText;\n                                }\n                                if (elem.innerHTML.includes("Process finished.")) {\n                                    halt__OBJ__(elem, "#333333");\n                                } else {\n                                    entrance___OBJ__ += 1;\n                                    console.log("req");\n                                    window.setTimeout("refresh__OBJ__()", 300); \n                                }\n                            }\n                            return xmlhttp.responseText;\n                        } else {\n                            errors___OBJ__ += 1;\n                            if (!entrance___OBJ__) {\n                                if (errors___OBJ__ < 6) {\n                                    entrance___OBJ__ += 1;\n                                    console.log("req");\n                                    window.setTimeout("refresh__OBJ__()", 300); \n                                } else {\n                                    halt__OBJ__(elem, "#994444");\n                                }\n                            }\n                        }\n                    }\n                }\n                xmlhttp.open("GET", "__FILE__", true);\n                xmlhttp.setRequestHeader("Cache-Control", "no-cache");\n                xmlhttp.send();     \n            }\n        }\n        \n        if (!entrance___OBJ__) {\n            entrance___OBJ__ += 1;\n            refresh__OBJ__(); \n        }\n        </script>\n\n        <p id="__OBJ__" style="font-size: 14px; background: #000000; padding: 10px; border: 3px; border-radius: 5px; color: white; ">\n        </p>\n        \n        </font>\n        <!--MD_END_FILTER-->\n        <!--MD_FROM_FILE __FILE__.md -->\n        \'\'\'.replace("__OBJ__", obj).replace("__FILE__", file)\n    if return_html_string:\n        return html_string\n    display(HTML(html_string))\n\n    \nclass TInteractiveLauncher:\n    tmp_path = "./interactive_launcher_tmp"\n    def __init__(self, cmd):\n        try:\n            os.mkdir(TInteractiveLauncher.tmp_path)\n        except:\n            pass\n        name = str(random.randint(0, 1e18))\n        self.inq_path = os.path.join(TInteractiveLauncher.tmp_path, name + ".inq")\n        self.log_path = os.path.join(TInteractiveLauncher.tmp_path, name + ".log")\n        \n        os.mkfifo(self.inq_path)\n        open(self.log_path, \'w\').close()\n        open(self.log_path + ".md", \'w\').close()\n\n        self.pid = os.fork()\n        if self.pid == -1:\n            print("Error")\n        if self.pid == 0:\n            exe_cands = glob.glob("../tools/launcher.py") + glob.glob("../../tools/launcher.py")\n            assert(len(exe_cands) == 1)\n            assert(os.execvp("python3", ["python3", exe_cands[0], "-l", self.log_path, "-i", self.inq_path, "-c", cmd]) == 0)\n        self.inq_f = open(self.inq_path, "w")\n        interactive_launcher_opened_set.add(self.pid)\n        show_log_file(self.log_path)\n\n    def write(self, s):\n        s = s.encode()\n        assert len(s) == os.write(self.inq_f.fileno(), s)\n        \n    def get_pid(self):\n        n = 100\n        for i in range(n):\n            try:\n                return int(re.findall(r"PID = (\\d+)", open(self.log_path).readline())[0])\n            except:\n                if i + 1 == n:\n                    raise\n                time.sleep(0.1)\n        \n    def input_queue_path(self):\n        return self.inq_path\n        \n    def close(self):\n        self.inq_f.close()\n        os.waitpid(self.pid, 0)\n        os.remove(self.inq_path)\n        # os.remove(self.log_path)\n        self.inq_path = None\n        self.log_path = None \n        interactive_launcher_opened_set.remove(self.pid)\n        self.pid = None\n        \n    @staticmethod\n    def terminate_all():\n        if "interactive_launcher_opened_set" not in globals():\n            globals()["interactive_launcher_opened_set"] = set()\n        global interactive_launcher_opened_set\n        for pid in interactive_launcher_opened_set:\n            print("Terminate pid=" + str(pid), file=sys.stderr)\n            os.kill(pid, signal.SIGKILL)\n            os.waitpid(pid, 0)\n        interactive_launcher_opened_set = set()\n        if os.path.exists(TInteractiveLauncher.tmp_path):\n            shutil.rmtree(TInteractiveLauncher.tmp_path)\n    \nTInteractiveLauncher.terminate_all()\n   \nyandex_metrica_allowed = bool(globals().get("yandex_metrica_allowed", False))\nif yandex_metrica_allowed:\n    display(HTML(\'\'\'<!-- YANDEX_METRICA_BEGIN -->\n    <script type="text/javascript" >\n       (function(m,e,t,r,i,k,a){m[i]=m[i]||function(){(m[i].a=m[i].a||[]).push(arguments)};\n       m[i].l=1*new Date();k=e.createElement(t),a=e.getElementsByTagName(t)[0],k.async=1,k.src=r,a.parentNode.insertBefore(k,a)})\n       (window, document, "script", "https://mc.yandex.ru/metrika/tag.js", "ym");\n\n       ym(59260609, "init", {\n            clickmap:true,\n            trackLinks:true,\n            accurateTrackBounce:true\n       });\n    </script>\n    <noscript><div><img src="https://mc.yandex.ru/watch/59260609" style="position:absolute; left:-9999px;" alt="" /></div></noscript>\n    <!-- YANDEX_METRICA_END -->\'\'\'))\n\ndef make_oneliner():\n    html_text = \'("В этот ноутбук встроен код Яндекс Метрики для сбора статистики использований. Если вы не хотите, чтобы по вам собиралась статистика, исправьте: yandex_metrica_allowed = False" if yandex_metrica_allowed else "")\'\n    html_text += \' + "<""!-- MAGICS_SETUP_PRINTING_END -->"\'\n    return \'\'.join([\n        \'# look at tools/set_up_magics.ipynb\\n\',\n        \'yandex_metrica_allowed = True ; get_ipython().run_cell(%s);\' % repr(one_liner_str),\n        \'display(HTML(%s))\' % html_text,\n        \' #\'\'MAGICS_SETUP_END\'\n    ])\n       \n\n');display(HTML(("В этот ноутбук встроен код Яндекс Метрики для сбора статистики использований. Если вы не хотите, чтобы по вам собиралась статистика, исправьте: yandex_metrica_allowed = False" if yandex_metrica_allowed else "") + "<""!-- MAGICS_SETUP_PRINTING_END -->")) #MAGICS_SETUP_END

<IPython.core.display.Javascript object>

# Лимиты в linux и отладка программ с помощью PTRACE

<p><a href="https://www.youtube.com/watch?v=???" target="_blank">
    <h3>Видеозапись семинара</h3> 
</a></p>


[Ридинг Яковлева](https://github.com/victor-yacovlev/mipt-diht-caos/tree/master/practice/exec-rlimit-ptrace) 

Сегодня в программе:
* <a href="#limits" style="color:#856024"> Лимиты в linux </a>  
* <a href="#ptrace" style="color:#856024"> Отладка программ с помощью PTRACE </a>  



## <a name="limits"></a> Лимиты в linux

In [5]:
!ulimit 

unlimited


In [2]:
!ulimit -a

time(seconds)        unlimited
file(blocks)         unlimited
data(kbytes)         unlimited
stack(kbytes)        8192
coredump(blocks)     0
memory(kbytes)       unlimited
locked memory(kbytes) 65536
process              7728
nofiles              4096
vmemory(kbytes)      unlimited
locks                unlimited
rtprio               0


1. RLIMIT_CPU - лимит на процессорное время, которое доступно процессу
  <br> `ulimit -t 1` - запретить процессам тратить больще 1 секунды процессорного времени.
1. RLIMIT_FSIZE - лимит на увеличение размера файлов.
  <br> `ulimit -f 100000` - ограничить максимальный размер создаваемого файла в 100 MB
1. RLIMIT_DATA - лимит на размер сегмента данных (глобальные переменные + куча).
  <br> `ulimit -d 48000` - ограничить максимальный размер сегмента данных в 48 MB
1. RLIMIT_STACK - лимит на размер стека ~ просто размер стека.
  <br> `ulimit -s 16384` - установить размер стека в 8 MB
1. RLIMIT_CORE - лимит на размер coredump'ов
  <br> `ulimit -c 0` - запрещаем создавать core файлы
  <br> `ulimit -c unlimited` - разрешаем неограниченно создавать core файлы (*****)
1. RLIMIT_RSS - лимит на потребление оперативной памяти (реально подгруженной в RAM). Работает только в определенных версиях linux. Так что не стоит закладываться на это.
1. RLIMIT_NPROC - лимит на количество процессов/потоков запущенных от екущего пользователя.
  <br> `ulimit -u 64` - запретить создавать больше 64 потоков/процессов.
1. RLIMIT_NOFILE - лимит на количество одновременно открытых файлов
  <br> `ulimit -n 5` - запретить открывать больше 5 файлов.
1. RLIMIT_AS - лимит на потребление виртуальной памяти.
  <br> `ulimit -v 100000` - ограничить размер используемой виртуальной памяти в 100 MB
...

In [58]:
%%cpp custom_stack.c
%run gcc custom_stack.c -o custom_stack.exe
%run ./custom_stack.exe 1000000


#include <sys/types.h>
#include <sys/wait.h>
#include <sys/time.h>
#include <sys/resource.h>
#include <stdint.h>
#include <stdio.h>
#include <stdlib.h>
#include <unistd.h>
#include <assert.h>
#include <inttypes.h>

    
void change_stack_size(uint64_t size, char** argv) {
    struct rlimit rlim;
    getrlimit(RLIMIT_STACK, &rlim);
    assert(RLIM_INFINITY == rlim.rlim_max || size <= rlim.rlim_max);
    
    //printf("size=%" PRIu64 " cur=%" PRIu64 "\n", size, rlim.rlim_cur);
    if (rlim.rlim_cur >= size) {
        return;
    }
    rlim.rlim_cur = size;
    setrlimit(RLIMIT_STACK, &rlim);

    pid_t pid = fork();
    assert(pid != -1);
    if (pid == 0) {
        execvp(argv[0], argv);
        perror("execvp: ");
        assert(0 && "exec");
    }
    
    int status;
    pid_t w = waitpid(pid, &status, 0);
    if (w == -1) {
        perror("waitpid");
        exit(-1);
    }
    if (WIFEXITED(status)) {
        //printf("exit status %" PRIu64 "\n", (uint64_t)WEXITSTATUS(status));
        exit(WEXITSTATUS(status));
    } else if (WIFSIGNALED(status)) {
        kill(getpid(), WTERMSIG(status));            
    }
}
    
uint64_t factorial(uint64_t n) {
    if (n <= 0) {
        return 1;
    }
    return (n % 13) * factorial(n - 1) % 13;
} 
    
int main(int argc, char** argv)
{
    assert(argc == 2);
    uint64_t n = strtoull(argv[1], NULL, 10);
    change_stack_size(n * 64 + 1000000, argv);
    printf("factorial(%" PRIu64 ") %% 13 == %" PRIu64 "\n", n, factorial(n));
}

Run: `gcc custom_stack.c -o custom_stack.exe`

Run: `./custom_stack.exe 1000000`

factorial(1000000) % 13 == 0


## <a name="ptrace"></a> Отладка программ с помощью PTRACE


Документация https://www.opennet.ru/base/dev/ptrace_guide.txt.html


In [42]:
%%cpp premoderate.c
%run gcc -m64 premoderate.c -o premoderate.exe
%run ./premoderate.exe echo "Vasya got 3 in math and 3 in russian"

#include <sys/types.h>
#include <sys/stat.h>
#include <sys/ptrace.h>
#include <sys/wait.h>
#include <sys/user.h>
#include <unistd.h>
#include <stdlib.h>
#include <stdio.h>
#include <fcntl.h>
#include <stdbool.h>
#include <asm/unistd.h>
#include <signal.h>
#include <string.h>
#include <errno.h>
#include <assert.h>

int check_code(int code, const char* file, int line, const char* text) {
    if (code == -1) {
        char buf[10000]; 
        sprintf(buf, "%s failed (%s:%d)", text, file, line); 
        perror(buf); 
        abort();
    }
    return code;
}
    
#define checked(call) check_code(call, __FILE__, __LINE__, #call)

static void premoderate_write_syscall(pid_t pid, struct user_regs_struct state)
{
    size_t orig_buf = state.rsi;   
    size_t size = state.rdx;
    char* buffer = calloc(size + sizeof(size_t), sizeof(*buffer));
    for (size_t i = 0; i < size; i += sizeof(size_t)) {
        *(size_t*)(buffer + i) = checked(ptrace(PTRACE_PEEKDATA, pid, orig_buf + i, NULL));
    }
    *(size_t*)(buffer + size - 1) = checked(ptrace(PTRACE_PEEKDATA, pid, orig_buf + size - 1, NULL));
    
    char* bad_char;
    while (bad_char = strchr(buffer, '3')) {
        *bad_char = '5';
        checked(ptrace(PTRACE_POKEDATA, pid, orig_buf + (bad_char - buffer), *(size_t*)bad_char));
    }
    free(buffer);
}

int main(int argc, char *argv[])
{
    pid_t pid = checked(fork());
    if (0 == pid) {
        checked(ptrace(PTRACE_TRACEME, 0, NULL, NULL));
        checked(execvp(argv[1], argv + 1));
    }  
    int wstatus = 0;
    checked(waitpid(pid, &wstatus, 0)); // Ждем пока дочерний процесс остановится на traceme
    do {
        checked(ptrace(PTRACE_SYSCALL, pid, NULL, NULL)); // Говорим, что будем ждать syscall и ждем
        checked(waitpid(pid, &wstatus, 0));
        if (WIFSTOPPED(wstatus)) {
            struct user_regs_struct state;
            checked(ptrace(PTRACE_GETREGS, pid, 0, &state));
            if (__NR_write == state.orig_rax) { 
                premoderate_write_syscall(pid, state);
            }              
        }
    } while (!WIFEXITED(wstatus));
}

Run: `gcc -m64 premoderate.c -o premoderate.exe`

Run: `./premoderate.exe echo "Vasya got 3 in math and 3 in russian"`

Vasya got 3 in math and 3 in russian


# Пример от меня, который может пригодиться при тестировании программ на надежность IO

In [99]:
%%cpp run_with_unreliable_io.c
%run gcc run_with_unreliable_io.c -o run_with_unreliable_io.exe

#include <sys/types.h>
#include <sys/stat.h>
#include <sys/ptrace.h>
#include <sys/wait.h>
#include <sys/user.h>
#include <unistd.h>
#include <stdlib.h>
#include <stdio.h>
#include <fcntl.h>
#include <stdbool.h>
#include <asm/unistd.h>
#include <signal.h>
#include <string.h>
#include <errno.h>
#include <time.h>


int check_code(int code, const char* file, int line, const char* text) {
    if (code == -1) {
        char buf[10000]; 
        sprintf(buf, "%s failed (%s:%d)", text, file, line); 
        perror(buf); 
        abort();
    }
    return code;
}
    
#define checked(call) check_code(call, __FILE__, __LINE__, #call)


int main(int argc, char *argv[])
{
    pid_t pid = checked(fork());
    if (0 == pid) {
        checked(ptrace(PTRACE_TRACEME, 0, NULL, NULL));
        checked(execvp(argv[3], argv + 3));
    }  
    srand(time(NULL));
    int always_partial = strtol(argv[1], NULL, 10);
    int enable_retryable_errors = strtol(argv[2], NULL, 10);
    
    int wstatus = 0;
    checked(waitpid(pid, &wstatus, 0));

    int reads_count = 0, writes_count = 0;
    do {
        checked(ptrace(PTRACE_SYSCALL, pid, NULL, NULL));
        checked(waitpid(pid, &wstatus, 0));
        if (WIFSTOPPED(wstatus)) {
            struct user_regs_struct state;
            checked(ptrace(PTRACE_GETREGS, pid, 0, &state));
            bool is_read = (__NR_read == state.orig_rax), is_write = (__NR_write == state.orig_rax);
            reads_count += is_read;
            writes_count += is_write;
            if (!(is_read || is_write) || (state.rdi == STDERR_FILENO) || (is_read && reads_count <= 2)) 
                continue;
              
            if (enable_retryable_errors && ((is_read && (reads_count & 1)) || (is_write && !(writes_count & 1)))) {
                unsigned long long old_rdx = state.rdx;
                unsigned long long old_rdi = state.rdi;
                state.rdx = 0;
                state.rdi = 100500; // not existing file descriptor
                checked(ptrace(PTRACE_SETREGS, pid, 0, &state));
                checked(ptrace(PTRACE_SINGLESTEP, pid, 0, 0));
                waitpid(pid, &wstatus, 0); // важно! делать wait
                state.rax = -EINTR;
                state.rdx = old_rdx;
                state.rdi = old_rdi;
                checked(ptrace(PTRACE_SETREGS, pid, 0, &state)); 
            } else if (always_partial || (random() % 3 != 0)) {
                unsigned long long old_rdx = state.rdx;
                if (state.rdx > 1) {
                    state.rdx = 1 + rand() % ((state.rdx + 4) / 5);
                    checked(ptrace(PTRACE_SETREGS, pid, 0, &state));
                } 
                checked(ptrace(PTRACE_SINGLESTEP, pid, 0, 0));
                waitpid(pid, &wstatus, 0);
                checked(ptrace(PTRACE_GETREGS, pid, 0, &state));
                // возвращаем как было, чтобы логика самой программы не поменялась
                state.rdx = old_rdx;
                checked(ptrace(PTRACE_SETREGS, pid, 0, &state)); 
            }          
        }
    } while (!WIFEXITED(wstatus));
}

Run: `gcc run_with_unreliable_io.c -o run_with_unreliable_io.exe`

In [108]:
!./run_with_unreliable_io.exe 1 0 echo "123 456 789 012 345 678 90"
!./run_with_unreliable_io.exe 1 1 echo "123 456 789 012 345 678 90"

123 456 789 012 345 678 90
123 45echo: write error


In [105]:
%%cpp unreliable_write.cpp
%run gcc unreliable_write.cpp -o unreliable_write.exe
%run ./unreliable_write.exe
%run ./run_with_unreliable_io.exe 1 0 ./unreliable_write.exe
%run ./run_with_unreliable_io.exe 1 1 ./unreliable_write.exe

#include <unistd.h>
#include <stdio.h>
#include <errno.h>
#include <string.h>

int main() {
    const char str[] = "Hello from C!\n";
    int written_p = printf("Reliable print: %s", str); fflush(stdout);
    fprintf(stderr, "Written %d bytes by printf. errno=%d, err=%s\n", written_p, errno, strerror(errno)); fflush(stderr);
  
    int written_w = write(1, str, sizeof(str));
    perror("write");
    fprintf(stderr, "Written %d bytes by write. errno=%d, err=%s\n", written_w, errno, strerror(errno)); fflush(stderr);
    
    return 0;
}

Run: `gcc unreliable_write.cpp -o unreliable_write.exe`

Run: `./unreliable_write.exe`

Reliable print: Hello from C!
Written 30 bytes by printf. errno=0, err=Success
Hello from C!
 write: Success
Written 15 bytes by write. errno=0, err=Success


Run: `./run_with_unreliable_io.exe 1 0 ./unreliable_write.exe`

Reliable print: Hello from C!
Written 30 bytes by printf. errno=0, err=Success
Hwrite: Success
Written 1 bytes by write. errno=0, err=Success


Run: `./run_with_unreliable_io.exe 1 1 ./unreliable_write.exe`

RelWritten -1 bytes by printf. errno=4, err=Interrupted system call
Hewrite: Interrupted system call
Written 2 bytes by write. errno=4, err=Interrupted system call


# Просто страшный код

Найдите ошибку и не повторяйте

In [116]:
%%cpp tmp.c --ejudge-style
%run gcc tmp.c -o tmp.exe
%run echo "asdf srfr" | ./tmp.exe
%run echo "asdf   srfr \n sdfvf" | ./tmp.exe

#include <stdio.h>
#include <stdlib.h>
#include <sys/wait.h>
#include <unistd.h>

int main()
{
    // setvbuf(stdin, NULL, _IONBF, 0); // а с этим может работать
    pid_t pid;
    int result = 0;
    while (1) {
        pid = fork();
        if (pid == 0) {
            char buffer[4097];
            int length = scanf("%s", buffer);
            return (length == EOF) ? 0 : 1;
        } else {
            int status;
            waitpid(pid, &status, 0);
            if (status == 0) {
                break;
            }
            result += WEXITSTATUS(status);
        }
    }
    printf("%d\n", result);
    return 0;
}

Run: `gcc tmp.c -o tmp.exe`

Run: `echo "asdf srfr" | ./tmp.exe`

1


Run: `echo "asdf   srfr \n sdfvf" | ./tmp.exe`

1
